# **High Value Customer Identification - Notebook A** 

Context
Typically e-commerce datasets are proprietary and consequently hard to find among publicly available data. However, The UCI Machine Learning Repository has made this dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail".

Content
"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

**Tópicos neste notebook:**

1 - Data Description

2 - Feature Engineering

3 - Filtragem de Variáveis

# Planejamento da Solução (IOT)

## Input - Entrada

1. Problema de negócio: selecionar os mais valiosos clientes para formar o programa de fidelidade **"INSIDERS"**
2. Conjunto de dados com as vendas de produtos durante 1 ano ( Dez 2010 to Dez 2011)

## Output - Saída

1. A indicação de pessoas para fazer parte do programa de fidelidade **"INSIDERS"**.

2. Relatório com as respostas para as seguintes perguntas:
    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Quantos clientes farão parte do grupo?
    - Quais as principais características desses clientes ?
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Quais as condições para uma pessoa ser removida do Insiders ?
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

## Tasks - Processo

1. **Quem são as pessoas elegíveis para participar do programa de Insiders ?**
    - O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
        - Faturamento:
            - Alto Ticket Médio
            - Alto LTV
            - Baixa Recência ou Alta Frequência ( tempo entre as compras )
            - Alto Basket Size ( quantidade média de produtos comprados )
            - Baixa probabilidade de Churn
            - Previsão alta de LTV
            - Alta propensão de compra

        - Custo:
            - Baixo número de devoluções

        - Experiência:  
            - Média alta de avaliações
            
            
2. **Quantos clientes farão parte do grupo?**
    - Número de clientes
    - % em relação ao total de clients
    
    
3. **Quais as principais características desses clientes ?**
    - Escrever os principais atributos dos clientes
        - Idade
        - País
        - Salário
        
    - Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
        - Vide acima
    
    
4. **Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?**
    - Calcular o faturamento total da empresa durante o ano.
    - Calcular o faturamento (%) apenas do cluster Insiders.
    
    
5. **Qual a expectativa de faturamento desse grupo para os próximos meses ?**
    - Cálculo do LTV do grupo Insiders
    - Séries Temporais ( ARMA, ARIMA, HoltWinter, etc )
    

6. **Quais as condições para uma pessoa ser elegível ao Insiders ?**
    - Qual o período de avaliação ?
    - O "desempenho" do cliente está próximo da média do cluster Insiders. 
    
    
7. **Quais as condições para uma pessoa ser removida do Insiders ?**
    - O "desempenho" do cliente não está mais próximo da média do cluster Insiders. 
    
    
8. **Qual a garantia que o programa Insiders é melhor que o restante da base ?**
    - Teste de Hipóteses
    - Teste A/B
    
    
9. **Quais ações o time de marketing pode realizar para aumentar o faturamento?**
    - Descontos
    - Preferências de escolha
    - Produtos exclusivos

# 0.0 Imports

In [80]:
import pandas as pd
import numpy as np
import inflection
import math
import re

from unidecode               import unidecode
from IPython.display         import Image, display

## 0.1 Helper Function

In [81]:
# Configurações Gerais
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## 0.2 Loading data

In [82]:
# Dataset treino
df_raw = pd.read_csv('../datasets/raw_datasets/data.csv', low_memory = False, encoding='ISO-8859-1')

# Removendo coluna vazia, devido ao csv ter vírgulas no final de cada linha
df_raw = df_raw.drop(['Unnamed: 8'], axis=1)

# **1.0 Data Description**

In [83]:
df1 = df_raw.copy()

In [84]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.00,United Kingdom


## 1.1 Rename Columns

In [85]:
# Renomeando as colunas usando a função underscore do inflection
df1.columns = [inflection.underscore(col) for col in df1.columns]
df1.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

## 1.2 Data Dimension

In [86]:
# Printando linhas/colunas
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Cols: 8


## 1.3 Data Types

In [87]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [88]:
'''
Existem 3 opções para tratar os NAs

1- Descartar linhas com valores ausentes (NA):
    Vantagem: rápido e fácil.
    Desvantagem (significativa): Pode levar à perda de informações importantes e prejudicar a performance do modelo se o conjunto de dados for pequeno.

2- Utilizar algoritmos de Machine Learning:
    Existem métodos para preencher valores ausentes (NA) que se baseiam no comportamento das colunas.
    Podemos utilizar métodos estatísticos como a média ou mediana, ou algorimos de Machine Learning para prever valores ausentes.
    Essa técnica é útil quando não se tem informações de negócio.

3- Entender a lógica de negócio:
    Compreendendo a lógica de negócio, é possível identificar o motivo dos valores ausentes e estabelecer regras para preenchê-los.  

Utilizarei a opção 3 para preencher os valores faltantes (NA) no próximo tópico.
'''

df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Fillout NA

In [89]:
# Preenchendo os NaN com lógica de negócio

# description
df1['description'] = df1['description'].fillna('no_description')

# customer_id
df1 = df1.dropna(subset=['customer_id'])

# Verificação da remoção dos NaN
df1.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

In [90]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.00,United Kingdom


## 1.6 Change Types

In [91]:
# Verificando os tipos de dados novamente após as alterações feitas no tópico 1.5, por boa prática.  
df1.dtypes

# Covertendo
df1['customer_id'] = df1['customer_id'].astype(int)
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])
df1.dtypes

C:\Users\andre\AppData\Local\Temp\ipykernel_20864\946814893.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])


invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id              int32
country                 object
dtype: object

O conjunto de dados inclui as seguintes informações:

| Coluna                     | Descrição                                           |
|----------------------------|-----------------------------------------------------|
| Invoice Number             | Identificador único de cada transação.              |
| Stock Code Product         | Código do item.                                    |
| Description Product        | Nome do item.                                      |
| Quantity                   | A quantidade de cada item comprado por transação.  |
| Invoice Date               | O dia em que a transação ocorreu.                   |
| Unit Price                 | Preço do produto por unidade.                      |
| Customer ID                | Identificador único do cliente.                     |
| Country                    | O nome do país onde o cliente reside.               |
| invoice_date_formatted     | Data formatada (dia/mês/ano).                      |
| month                      | Número do mês.                                     |


## 1.7 Minority Changes (map data)

In [92]:
df1 = df1.drop_duplicates()

# **2.0 Feature Engineering**

In [93]:
df2 = df1.copy()

## **2.1 MindMap Hypotheses**

<p>O Mindmap de Hipóteses nos ajuda a criar hipóteses sobre o fenômeno estudado. Tais hipóteses geradas serão investigadas na EDA (Análise Exploratória de Dados).</p>
<p>Na imagem abaixo encontra-se o Mindmap de hipóteses, onde ao centro, o 'Preço do Veículo' é o fenômeno a ser investigado e está rodeado por suas entidades 'Vendedor', 'Veículo' e 'Documentação do Veículo', seguidas por seus atributos.

In [94]:
# Image('../images/mindmap_.png')

NOTA: No cotidiano de uma empresa, as etapas iniciais são compostas por:

1- Investigação individual: Nesta fase, eu, como profissional responsável pelo projeto, realizo investigações e análises preliminares, criando incialmente meu próprio mindmap.

2- Após a investigação inicial, compartilho minhas descobertas com a área de negócio, onde os stakeholders avaliam e contribuem com suas perspectivas e ideias.

3- Com base nas ideias reunidas tanto por mim quanto pela área de negócio, a equipe técnica e de análise de dados entra em ação. Juntos, verificamos quais dados estão disponíveis no banco de dados para implementar as ideias discutidas anteriormente.

Obs.: Os atributos com asterisco não estão presentes no dataset, mas é interessante investigar, pois pode contribuir para entender mais sobre o problema de negócio e talvez melhorar a performance do modelo de machine learning.

## **2.2 Hypotheses Creation**

### 2.2.1 Hipóteses do Veículo

**1.** Carros mais antigos possuem um preço menor.

**2.** Carros com maior valor de hodômetro possuem valores menores.

**3.** Carros na cor branca possuem preços maiores.

**4.** Carros com cambio manual e com valores maiores de hodometro, possuem preços menores.

### 2.2.2 Hipóteses da Documentação

**1.** Carros com garantia de fábrica possuem preços maiores.

**2.** Carros com revisões feitas em dia e nas concessionárias, possuem preços maiores.


### 2.2.3 Hipóteses do Vendedor

**1.** Carros vendidos por pessoa física possuem preços menores.

**2.** Carros da região Sudeste possuem preços maiores em média.

**3.** Vendedores que aceitam troca possuem carros com preços menores.

**4.** Carros populares de baixo padrão são mais vendidos por pessoas físicas.

## **2.3 Lista Final de Hipóteses**

**1.** Carros mais antigos possuem um preço menor.

**2.** Carros com maior valor de hodômetro possuem valores menores.

**3.** Carros na cor branca possuem preços maiores.

**4.** Carros com cambio manual e com valores maiores de hodometro, possuem preços menores.

**5.** Carros com garantia de fábrica possuem preços maiores.

**6.** Carros com revisões feitas em dia e nas concessionárias, possuem preços maiores.

**7.** Carros vendidos por pessoa física possuem preços menores.

**8.** Carros da região Sudeste possuem preços maiores em média.

**9.** Vendedores que aceitam troca possuem carros com preços menores.

**10.** Carros populares de baixo padrão são mais vendidos por pessoas físicas.

## **2.4 Feature Engineering**

In [95]:
df2.describe().T

,count,mean,min,25%,50%,75%,max,std
quantity,401603.00,12.18,-80995.00,2.00,5.00,12.00,80995.00,250.28
invoice_date,401603,2017-07-07 22:53:53.288595712,2016-11-29 00:00:00,2017-04-04 00:00:00,2017-07-27 00:00:00,2017-10-18 00:00:00,2017-12-07 00:00:00,NaN
unit_price,401603.00,3.47,0.00,1.25,1.95,3.75,38970.00,69.76
customer_id,401603.00,15281.16,12346.00,13939.00,15145.00,16784.00,18287.00,1714.01


In [96]:
df2.head(1)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,17850,United Kingdom


In [97]:
# Faturamento por linha 
df2['faturamento'] = df2['quantity'] * df2['unit_price']

In [98]:
# Função para separar letras e números
def separar_letras_e_numeros(codigo):
    padrao = re.match(r'([A-Za-z]*)(\d*)', codigo)
    
    if padrao:
        letras = padrao.group(1)  # Converta para maiúsculas se necessário
        numeros = padrao.group(2)
        return letras, numeros
    else:
        return None, None

# Aplicar a função a cada valor na coluna 'codigo'
df2[['invoice_letter', 'invoce_number']] = df2['invoice_no'].apply(lambda x: pd.Series(separar_letras_e_numeros(x)))

# Verificando se todos os valores da coluna quantidade são negativos.
df2[(df2['invoice_letter'] == 'C') & (df2['quantity'] > 0)].shape[0]

0

In [141]:
# Filtrando dataset  -------------------

# Produtos cancelados 
df2c = df2[df2['invoice_letter'] == 'C']

# Produtos comprados 
df2b = df2[df2['invoice_letter'] != 'C']

prct_cancel = round(df2c.shape[0] / df2b.shape[0], 2) * 100
print(f'Os cancelamentos representam {prct_cancel}% do total de vendas')

Os cancelamentos representam 2.0% do total de vendas


In [142]:
# Recencia, valor médio entre uma 
df2['invoice_date'].max()

Timestamp('2017-12-07 00:00:00')

In [143]:
aux = df2b.groupby('customer_id').agg({ 'faturamento': 'sum', 'invoice_no':'count'}).reset_index()
aux.columns = ['cliente', 'total_valor', 'total_transacoes']
aux['ticket_medio'] = aux['total_valor'] / aux['total_transacoes']
aux

,cliente,total_valor,total_transacoes,ticket_medio
0,12346,77183.60,1,77183.60
1,12347,4310.00,182,23.68
2,12348,1797.24,31,57.98
3,12349,1757.55,73,24.08
4,12350,334.40,17,19.67
...,...,...,...,...
4334,18280,180.60,10,18.06
4335,18281,80.82,7,11.55
4336,18282,178.05,12,14.84
4337,18283,2045.53,721,2.84


In [144]:
# Data de ultima compra de cada cliente
aux1 = df2b.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
aux1.rename(columns={'invoice_date': 'ultima_compra'}, inplace=True)

# Data max do dataset df2b inteiro
aux1['data_max'] = df2b['invoice_date'].max()

# Calculando a diferença para achar a data de ultima compra do cliente em dias
aux1['dias_desde_ultima_compra'] = (aux1['data_max'] - aux1['ultima_compra']).dt.days
aux1

,customer_id,ultima_compra,data_max,dias_desde_ultima_compra
0,12346,2017-01-16,2017-12-07,325
1,12347,2017-12-05,2017-12-07,2
2,12348,2017-09-23,2017-12-07,75
3,12349,2017-11-19,2017-12-07,18
4,12350,2017-01-31,2017-12-07,310
...,...,...,...,...
4334,18280,2017-03-05,2017-12-07,277
4335,18281,2017-06-10,2017-12-07,180
4336,18282,2017-11-30,2017-12-07,7
4337,18283,2017-12-04,2017-12-07,3


In [169]:
# Agrupando todos os chamados de cada cliente e utilizando o set para pegar apenas uma data unica, estava se
# repetindo as datas, pois cada linha do dataset é um produto, se o cliente comprou mais de um produto no 
# mesmo dia resultava em várias datas iguais na coluna invoice_date com código de transação invoice_no repetido
result = df2b.groupby('customer_id')['invoice_date'].agg(lambda x: list(set(x))).reset_index()
result.columns = ['customer_id', 'unique_invoice_dates']
result

,customer_id,unique_invoice_dates
0,12346,[2017-01-16 00:00:00]
1,12347,"[2017-07-31 00:00:00, 2016-12-05 00:00:00, 201..."
2,12348,"[2017-04-03 00:00:00, 2017-01-23 00:00:00, 201..."
3,12349,[2017-11-19 00:00:00]
4,12350,[2017-01-31 00:00:00]
...,...,...
4334,18280,[2017-03-05 00:00:00]
4335,18281,[2017-06-10 00:00:00]
4336,18282,"[2017-08-03 00:00:00, 2017-11-30 00:00:00]"
4337,18283,"[2017-10-25 00:00:00, 2017-05-21 00:00:00, 201..."


In [176]:
def recencia_media_3(dates_list, max_date):

    # Colocando lista em ordem descrescente 
    sorted_dt_list = sorted(dates_list, reverse=True)
    
    # Encontrado o intervalo de tempo entre compras, em dias

    if len(dates_list) == 1:
        c1 = (max_date - sorted_dt_list[0]).days
        return c1
    
    if len(dates_list) == 2:
        c1 = (max_date - sorted_dt_list[0]).days
        c2 = (sorted_dt_list[0] - sorted_dt_list[1]).days
        return (c1+c2) / 2

    if len(dates_list) >= 3:
        c1 = (max_date - sorted_dt_list[0]).days
        c2 = (sorted_dt_list[0] - sorted_dt_list[1]).days
        c3 = (sorted_dt_list[1] - sorted_dt_list[2]).days
        return (c1+c2+c3) / 3

In [177]:
# Recencia Média em dias das 3 ultimas compras de cada cliente
max_date = df2b['invoice_date'].max()
result['recencia_media_3'] = result['unique_invoice_dates'].apply(recencia_media_3, args=(max_date,))

# Recencia Média em dias de todas as compras de cada cliente


In [178]:
result

,customer_id,unique_invoice_dates,recencia_media_3
0,12346,[2017-01-16 00:00:00],325.00
1,12347,"[2017-07-31 00:00:00, 2016-12-05 00:00:00, 201...",43.00
2,12348,"[2017-04-03 00:00:00, 2017-01-23 00:00:00, 201...",106.00
3,12349,[2017-11-19 00:00:00],18.00
4,12350,[2017-01-31 00:00:00],310.00
...,...,...,...
4334,18280,[2017-03-05 00:00:00],277.00
4335,18281,[2017-06-10 00:00:00],180.00
4336,18282,"[2017-08-03 00:00:00, 2017-11-30 00:00:00]",63.00
4337,18283,"[2017-10-25 00:00:00, 2017-05-21 00:00:00, 201...",5.33


In [167]:
import pandas as pd

# Criando dois objetos Timestamp
data1 = pd.Timestamp('2022-01-01')
data2 = pd.Timestamp('2022-01-10')

# Calculando a diferença em dias
diferenca_em_dias = (data2 - data1).days

# Imprimindo a diferença
print(diferenca_em_dias)


9


In [159]:
df2b[(df2b['customer_id'] == 12347) & (df2b['invoice_no'] == '537626')]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,faturamento,invoice_letter,invoce_number
14938,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2016-12-05,2.10,12347,Iceland,25.20,,537626
14939,537626,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2016-12-05,4.25,12347,Iceland,17.00,,537626
14940,537626,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2016-12-05,3.25,12347,Iceland,39.00,,537626
14941,537626,22492,MINI PAINT SET VINTAGE,36,2016-12-05,0.65,12347,Iceland,23.40,,537626
14942,537626,22771,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626
14943,537626,22772,PINK DRAWER KNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626
14944,537626,22773,GREEN DRAWER KNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626
14945,537626,22774,RED DRAWER KNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626
14946,537626,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626
14947,537626,22805,BLUE DRAWER KNOB ACRYLIC EDWARDIAN,12,2016-12-05,1.25,12347,Iceland,15.00,,537626


In [140]:
diferencas = df2b.groupby('customer_id')['invoice_date'].mean().reset_index()
diferencas

,customer_id,invoice_date
0,1,2022-01-05 08:00:00
1,2,2022-02-03 00:00:00
2,3,2022-03-01 00:00:00


In [135]:
# Encontrar a data máxima para todo o DataFrame
data_max = df2b['invoice_date'].max()

# Calcular a diferença em dias entre a data máxima e as três últimas transações de cada cliente
diferencas = df2b.groupby('customer_id')['invoice_date'].apply(lambda x: (data_max - x.nlargest(3).iloc[::-1]).dt.days)

# Calcular a média das diferenças em dias
media_3_ultimas_compras_dias = diferencas.groupby('customer_id').mean().reset_index()

# Renomear a coluna resultante
media_3_ultimas_compras_dias.columns = ['customer_id', 'media_3_ultimas_compras_dias']

print(media_3_ultimas_compras_dias)


   customer_id  media_3_ultimas_compras_dias
0            1                         54.67
1            2                         26.00
2            3                          0.00


In [137]:
df2b.groupby('customer_id')['invoice_date'].mean().reset_index()

,customer_id,invoice_date
0,1,2022-01-05 08:00:00
1,2,2022-02-03 00:00:00
2,3,2022-03-01 00:00:00


In [133]:
media_diff_dias

0.0

In [128]:
df2b.sample()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,faturamento,invoice_letter,invoce_number
410526,572106,22566,FELTCRAFT HAIRBAND PINK AND PURPLE,12,2017-10-18,0.85,15058,United Kingdom,10.20,,572106


In [122]:
# Ordenar o DataFrame por 'customer_id' e 'invoice_date' de forma descendente
df2b = df2b.sort_values(['customer_id', 'invoice_date'], ascending=[True, False])
# # Criar uma coluna 'rank' para representar a ordem de compra por cliente
df2b['rank'] = df2b.groupby('customer_id').cumcount() + 1

# # Filtrar apenas as últimas três transações de cada cliente
ultimas_transacoes = df2b[df2b['rank'] <= 3]

# # Calcular a diferença em dias entre as transações consecutivas
ultimas_transacoes['dias_entre_transacoes'] = (ultimas_transacoes.groupby('customer_id')['invoice_date'].diff()).dt.days
df2b

# # Calcular a média do tempo em dias
media_dias_entre_transacoes = ultimas_transacoes.groupby('customer_id')['dias_entre_transacoes'].mean().reset_index()
media_dias_entre_transacoes

C:\Users\andre\AppData\Local\Temp\ipykernel_20864\2682881464.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ultimas_transacoes['dias_entre_transacoes'] = (ultimas_transacoes.groupby('customer_id')['invoice_date'].diff()).dt.days


,customer_id,dias_entre_transacoes
0,12346,NaN
1,12347,0.00
2,12348,0.00
3,12349,0.00
4,12350,0.00
...,...,...
4334,18280,0.00
4335,18281,0.00
4336,18282,0.00
4337,18283,0.00


In [ ]:
df2b['invoice_date'].max()

'31/10/2017'

In [ ]:
# Calcular a última compra para cada cliente
ultima_compra_por_cliente = df2b.groupby('customer_id')['invoice_date'].max().reset_index()

# Renomear as colunas
ultima_compra_por_cliente.columns = ['customer_id', 'ultima_compra_em']

# Calcular a data máxima de todo o DataFrame
data_maxima = df2b['invoice_date'].max()

# Converter para o tipo de dados datetime
data_maxima = pd.to_datetime(data_maxima)

# Criar DataFrame auxiliar com 'data_max' igual para todos os clientes
aux1 = pd.DataFrame({'customer_id': ultima_compra_por_cliente['customer_id'], 'ultima_compra_em': ultima_compra_por_cliente['ultima_compra_em'], 'data_max': data_maxima})

# Calcular a diferença em dias
aux1['dias_desde_ultima_compra'] = (aux1['data_max'] - aux1['ultima_compra_em']).dt.days


C:\Users\andre\AppData\Local\Temp\ipykernel_20864\3434653974.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_maxima = pd.to_datetime(data_maxima)
C:\Users\andre\AppData\Local\Temp\ipykernel_20864\3434653974.py:17: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  aux1['dias_desde_ultima_compra'] = (aux1['data_max'] - aux1['ultima_compra_em']).dt.days


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'str'

In [ ]:
# # Encontrar a última data do dataset
# aux['data_max'] = df2b['invoice_date'].max()
# aux

# Calcular a diferença em dias entre a data máxima e a última compra de cada cliente
aux1 = df2b.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
aux1.columns = ['customer_id', 'ultima_compra_em']
aux1['data_max'] = df2b['invoice_date'].max()

aux1['data_max'] = pd.to_datetime(aux1['data_max'])
aux1['ultima_compra_em'] = pd.to_datetime(aux1['ultima_compra_em'])

aux1['dias_desde_ultima_compra'] = (aux1['data_max'] - aux1['ultima_compra_em']).dt.days
aux1
# # aux.head()

C:\Users\andre\AppData\Local\Temp\ipykernel_20864\3021160753.py:10: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  aux1['data_max'] = pd.to_datetime(aux1['data_max'])
C:\Users\andre\AppData\Local\Temp\ipykernel_20864\3021160753.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  aux1['ultima_compra_em'] = pd.to_datetime(aux1['ultima_compra_em'])


,customer_id,ultima_compra_em,data_max,dias_desde_ultima_compra
0,12346,2017-01-16,2017-10-31,288
1,12347,2017-07-31,2017-10-31,92
2,12348,2017-09-23,2017-10-31,38
3,12349,2017-11-19,2017-10-31,-19
4,12350,2017-01-31,2017-10-31,273
...,...,...,...,...
4334,18280,2017-03-05,2017-10-31,240
4335,18281,2017-06-10,2017-10-31,143
4336,18282,2017-11-30,2017-10-31,-30
4337,18283,2017-11-28,2017-10-31,-28


In [ ]:
df2b.groupby('customer_id')['invoice_date']

In [ ]:
df2.sample()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,invoice_letter,invoce_number
299092,563078,21929,JUMBO BAG PINK VINTAGE PAISLEY,10,09/08/2017,2.08,15955,United Kingdom,,563078


In [ ]:
df2['invoice_no']

In [ ]:
# Faturamento por linha 
df2['faturamento'] = df2['quantity'] * df2['unit_price']

# Monetary
df_monetary = df2[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

# Recency - Last day purchase
# **inverter função**
df_recency = df2[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_recency['recency_days'] = df_recency['recency_days'].apply( lambda x: 1 / x if x != 0 else 0)
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

# Frequency
df_freq = df2[['customer_id', 'invoice_no']].drop_duplicates().groupby( 'customer_id' ).count().reset_index()
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )

# Avg Ticket
df_avg_ticket = df2[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
# print(f"{df2.isna().sum().sum()} valores NAN encontrados.")

0 valores NAN encontrados.
0 valores NAN encontrados.
0 valores NAN encontrados.
0 valores NAN encontrados.
0 valores NAN encontrados.
0 valores NAN encontrados.


In [ ]:
df2.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

In [ ]:
df2[df2['unit_price'] < 0]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


# **3.0 Filtragem de Variáveis**

In [ ]:
df3 = df2.copy()

## 3.1 Filtragem das Colunas

In [ ]:
# Filtragem/remoção/drop de colunas com valores constantes ou 100% nulos.
cols_drop = df3.nunique()[df3.nunique() <= 1].index.tolist() # Dropando 
df3 = df3.drop(cols_drop, axis=1)

# Removido as colunas 'veiculo_alienado' e 'elegivel_revisao', pois não fornecem informações para o modelo de machine learning, 
# já que todos os valores, em ambas as colunas possuiam um único valor.


# Remoção da coluna versão, devido a feature engineering aplicada nela
df3 = df3.drop(['versao'], axis=1)


print(f"df2: {df2.shape[1]} colunas.\ndf3: {df3.shape[1]} colunas.")
print(f"{df2.shape[1] - df3.shape[1]} colunas foram removidas.")

df2: 36 colunas.
df3: 33 colunas.
3 colunas foram removidas.


## 3.2 Exportando dataset processado

In [ ]:
df3.to_csv('../datasets/cooked_datasets/df3_processed.csv', index=False)